In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
CI = 'TRAVIS' in os.environ
if CI:
    os.environ["NUMBA_DISABLE_JIT"] = "1"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup, default_opt_set, option_string
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21, modified_Smolarkiewicz_Rasch_r0
from MPyDATA_examples.Olesik_et_al_2020.analysis  import rel_disp

In [4]:
GCs = np.linspace(.05,.95, 3) if CI else np.linspace(.05,.95, 7)
nrs = np.array([2**n for n in (range(7, 10) if CI else range(7,15))], dtype=int)

In [5]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [6]:
const = min(np.log2(1/nrs)) - 1


def phi(cour):
    return cour * np.pi / 2

def rho(n):
    return np.log2(1/n) - const


def polar_plot(nr, cour, values, name):
    theta_array = phi(cour)
    r_array = rho(nr)

    X, Y = np.meshgrid(theta_array, r_array)
    Z = np.array(list(values)).reshape(len(r_array), len(theta_array))

    min_val = -26
    max_val = -2

    amplitude = max_val - min_val
    if name == 'log$_2$(err)':
        levels = np.linspace(
            min_val,
            max_val,
            int(amplitude + 1)
        )
    else:
        levels = 7

    ax = plt.subplot(111, projection='polar')
    cnt = ax.contourf(X, Y, Z, levels, cmap='jet')
    plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
    ax.scatter(X, Y, alpha=.8, s=10)
    legend = plt.colorbar(cnt, ax=ax, pad=0.1)
    legend.set_label(r'$log_2(Err_{L2})$', rotation=90)
    ax.set_thetalim(min(theta_array),max(theta_array))
    ax.set_rlim(0, max(r_array))
    
    ticks = r_array
    ax.set_yticks(ticks)
    ax.set_yticklabels("$2^{" + f"{int(-tick - const):d}" + "}$" for tick in ticks)
    
    ax.set_thetagrids(cour * 90, tuple(f"{c:.2g}" for c in cour))
    ax.grid(True)
    ax.set_xlabel(r"$nx$", labelpad=18)
    ax.annotate(r'$C$', xy=(.8, .85), xycoords='axes fraction')

In [7]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(1)
    simulation.step(simulation.out_steps[-1]-1)
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True,
                r=simulation.r.magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    psi = simulation.psi
    g_factor = simulation.g_factor
    dp_dr = simulation.dp_dr
    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    dif = analytical-numerical
    measure_h_2 =  (1/t * np.sqrt( 1/len(dif) * dif.dot(dif))).magnitude
  
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    error_g = np.log2(Smolarkiewicz_Grabowski_1990_eq21(g_factor * psi.magnitude, g_factor * analytical.magnitude / dp_dr, t.magnitude))
    error_r0 = np.log2(modified_Smolarkiewicz_Rasch_r0(psi.magnitude, analytical.magnitude /dp_dr, t.magnitude, g_factor))
    return setup.nr, GC, error

In [ ]:
for opt in default_opt_set.values():
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2]}
    opt=option_string(str(opt))
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  2.6min
